<a href="https://colab.research.google.com/github/YDKDevelop/yquantum-2025/blob/main/new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

As the main goal of this challenge, you are expected to create a hash function based on quantum simulation. Your hash function's performance evaluation will be based on the following criteria:

1. Output determinism
2. Preservation of entropy
3. Computational difficulty
4. Preimage resistance
5. Collision resistance
6. Computational feasibility
7. Computation time
8. Purely quantum hashing

Their meaning will be demonstrated on a simple (and very bad) hash function.

In [15]:
!pip install qiskit
!pip install qiskit matplotlib

In [22]:
import math
import numpy as np
import matplotlib.pyplot as plt
import random
import timeit
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector
from qiskit.quantum_info.operators import Pauli
from qiskit.visualization import plot_bloch_multivector


# Constants
NUM_QUBITS = 16
NUM_LAYERS = 2             # From the paper Designing Hash and Encryption Engines using Quantum Computing
TOTAL_BITS = 16            # Number of bits per expectation value
FRACTION_BITS = 15         # Precision within each fixed-point

def toFixed(x: float) -> int:
    """Convert float to fixed-point integer."""
    fraction_mult = 1 << FRACTION_BITS
    return int(x * fraction_mult + (0.5 if x >= 0 else -0.5))

def bytes_to_angles(input_bytes: bytes) -> list[float]:
    """Convert 256-bit input (32 bytes) into 2π-scaled rotation angles."""
    assert len(input_bytes) == 32, "Input must be 256 bits (32 bytes)"
    return [(b / 255.0) * 2 * math.pi for b in input_bytes]

def build_quantum_hash_circuit(angles: list[float]) -> QuantumCircuit:
    """Build a 16-qubit, 4-layer PQC circuit using parameterized rotations."""
    qc = QuantumCircuit(NUM_QUBITS)

    angle_chunks = np.array_split(angles, NUM_LAYERS)

    for layer_idx, chunk in enumerate(angle_chunks):
        for i in range(NUM_QUBITS):
            angle = chunk[i % len(chunk)]
            if layer_idx % 2 == 0:
                qc.rx(angle, i)
            else:
                qc.ry(angle, i)

        # Entanglement pattern (nearest neighbor + CZ cross links)
        for i in range(NUM_QUBITS - 1):
            qc.cx(i, i + 1)
        if layer_idx % 2 == 0:
            for i in range(0, NUM_QUBITS - 2, 2):
                qc.cz(i, i + 2)

    return qc

def quantum_hash(input_bytes: bytes) -> bytes:
    """Compute a 256-bit quantum hash of 32-byte input."""
    angles = bytes_to_angles(input_bytes)
    qc = build_quantum_hash_circuit(angles)
    sv = Statevector.from_instruction(qc)

    # Measure expectation values (Z-basis)
    exps = [sv.expectation_value(Pauli("Z"), [i]).real for i in range(NUM_QUBITS)]
    fixed_exps = [toFixed(exp) for exp in exps]

    # Convert to 256-bit hash (16 values × 16 bits = 256 bits)
    output_bytes = []
    for fixed in fixed_exps:
        for j in range(TOTAL_BITS // 8):
            output_bytes.append((fixed >> (8 * j)) & 0xFF)

    return bytes(output_bytes)

# Example: Hash for input 222
num = 222
input_bytes = num.to_bytes(32, byteorder='big')
hash_output = quantum_hash(input_bytes)
print("Hash for input 222:", hash_output.hex())
print("Total Time:", timeit.timeit(lambda: quantum_hash(input_bytes), number=1))

# # ------------------------------
# # Generate a distribution from many random inputs
# # ------------------------------

# num_samples = 100  # number of random inputs to generate
# all_fixed_values = []  # collect each of the 16 fixed-point values for each hash

# for _ in range(num_samples):
#     # Generate a random 32-byte input.
#     input_bytes = bytes(random.randint(0, 255) for _ in range(32))
#     # Compute hash (we only need the fixed_exps; you can also call quantum_hash() if desired)
#     angles = bytes_to_angles(input_bytes)
#     qc = build_quantum_hash_circuit(angles)
#     sv = Statevector.from_instruction(qc)
#     exps = [sv.expectation_value(Pauli("Z"), [i]).real for i in range(NUM_QUBITS)]
#     fixed_exps = [toFixed(exp) for exp in exps]
#     all_fixed_values.extend(fixed_exps)

# # Plotting the histogram:
# plt.figure(figsize=(10,6))
# plt.hist(all_fixed_values, bins=50, edgecolor='black')
# plt.xlabel("Fixed-point value (per qubit expectation)")
# plt.ylabel("Frequency")
# plt.title("Distribution of Fixed-point Values from Quantum Hash Outputs\n(Collected over {} random inputs)".format(num_samples))
# plt.show()


Hash for input 222: 008000800080008000800080008000800080008000800080008000800080f757
Total Time: 0.07459608299996034


In [17]:
# import math
# from qiskit import QuantumCircuit
# from qiskit.circuit import Parameter
# from qiskit.quantum_info import Statevector
# from qiskit.quantum_info.operators import Pauli

# TOTAL_BITS = 16
# FRACTION_BITS = 15

# # convert a float expectation to fixed-point
# def toFixed(x: float) -> int:
#     fraction_mult = 1 << FRACTION_BITS
#     return int(x * fraction_mult + (0.5 if x >= 0 else -0.5))


# NUM_QUBITS = 16
# NUM_LAYERS = 2

# # build the parameterized quantum circuit.
# qc = QuantumCircuit(NUM_QUBITS)
# params = []
# for l in range(NUM_LAYERS):
#     # add parameterized RY rotation gates
#     for i in range(NUM_QUBITS):
#         theta = Parameter(f"theta_ry_{l}_{i}")
#         params.append(theta)
#         qc.ry(theta, i)
#     # add parameterized RX rotation gates
#     for i in range(NUM_QUBITS):
#         theta = Parameter(f"theta_rz_{l}_{i}")
#         params.append(theta)
#         qc.rz(theta, i)
#     # add CNOT entangling gates
#     for i in range(NUM_QUBITS - 1):
#         qc.cx(i, i + 1)
# num_params = len(params)


# # Quantum simulation portion of the qhash
# # x - 256-bit byte array
# # returns the hash value as a 256-bit byte array
# def qhash(x: bytes) -> bytes:
#     # create a dictionary mapping each parameter to its value.
#     param_values = {}
#     for i in range(num_params):
#         # extract a nibble (4 bits) from the input
#         nibble = (x[i // 2] >> (4 * (1 - (i % 2)))) & 0x0F
#         # scale it to use as a rotation angle parameter
#         value = nibble * math.pi / 8
#         param_values[params[i]] = value

#     # bind the parameters to the circuit.
#     bound_qc = qc.assign_parameters(param_values)

#     # prepare the state vector from the bound circuit
#     sv = Statevector.from_instruction(bound_qc)
#     # calculate the qubit expectations on the Z axis
#     exps = [sv.expectation_value(Pauli("Z"), [i]).real for i in range(NUM_QUBITS)]
#     # convert the expectations to the fixed-point values
#     fixed_exps = [toFixed(exp) for exp in exps]

#     # pack the fixed-point results into a byte list.
#     data = []
#     for fixed in fixed_exps:
#         for i in range(TOTAL_BITS // 8):
#             data.append((fixed >> (8 * i)) & 0xFF)

#     return bytes(data)

The following hash function uses one qubit per one byte of input and applies the RX gates with the angles proportional to the bytes' values:

In [18]:
# from qiskit import QuantumCircuit
# from qiskit.quantum_info import Pauli, Statevector
# import numpy as np

# def simple_quantum_hash(input_bytes: bytes):
#     num_qubits = len(input_bytes)
#     qc = QuantumCircuit(num_qubits)
#     for i in range(num_qubits):
#         angle = (input_bytes[i] / 255) * np.pi  # scale to [0, π]
#         qc.rx(angle, i)

#     sv = Statevector.from_instruction(qc)
#     exp_vals = [sv.expectation_value(Pauli("Z"), [i]).real for i in range(num_qubits)]

#     # Map each expectation value from [-1, 1] to an 8-bit integer in [0, 255].
#     output_bytes = bytearray([min(int(((val + 1) / 2) * 256), 255) for val in exp_vals])

#     return output_bytes

At the very least, this function meets 2 of our most straightforward requirements. Firstly, it consistently produces the same output for a given input, satisfying the 'Output determinism' constraint, and, secondly, it does not use any classical hashing algorithms as required by the 'Purely classical hashing' point.

Let's now see what output our hash function produces for some simple inputs:

In [19]:
# print(list(simple_quantum_hash(bytes(range(0, 260, 20)))))

As you might've already deduced from the function definition, it basically calculates shifted and quantized cosine value for each of the input bytes. Unfortunately this fails to pass the 'Computational difficulty' requirement, as it is trivial to implement the same function without using any quantum simulators and such function would run in linear time with respect to the input length. This also makes finding the preimage of a given hash output an easy task, making it a cryptographically-poor hash function that is non-compliant with the 'Preimage resistance' criteria.

Since the cosine itself is a bijection from the domain of [0, π] to the codomain of [-1, 1] the hash collisions can only be possible due to the quantization. The easiest way to find such collisions is to look at the values closest to the minimum and maximum of a single byte range, where the derivative of the cosine is small:

In [20]:
# print(list(simple_quantum_hash(bytes(range(0, 20)))))
# print(list(simple_quantum_hash(bytes(range(236, 256)))))

As you can see, there is plenty of byte collisions to be found in those regions. As a result, using this hash algorithm would result in a higher collision rate when compared to its classical counterparts, making it unsuitable for production applications and failing the 'Collision resistance' constraint.

The last non-trivial requirement is the 'Preservation of entropy', which our function, yet again, does not pass. The reason for that is simple - the cosine is not a linear function. That means that feeding a large set of randomly-generated inputs to our function would result in its output bytes being more highly concentrated towards their maximum and minimum values (as was previously demonstrated) and failing to behave like a sample from a random distribution over the function's codomain. Furthermore, some of the byte values are not possible to obtain at all due to the quantization:

In [21]:
# print(list(simple_quantum_hash(bytes(range(120, 135)))))

Lastly, the provided hash function does not pass the 'Computational feasibility' requirement, since it would require to simulate a 32-qubit circuit to calculate a 256-bit hash, which is not feasible to do on general-purpose hardware. This also makes the 'Computation time' criteria irrelevant, even thogh the function only employs one gate per qubit which would lead to good performance if the number of qubits was decreased.